In [1]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from os import path
import jieba

In [ ]:
!head -2 ./data/train.csv

In [ ]:
# in_f = open('./data/train.csv')
# count = 0
# while True: 
#     count += 1
#     # Get next line from file 
#     line = in_f.readline() 
#     # if line is empty 
#     # end of file is reached 
#     if not line or count > 1: 
#         break
#     print(f"Line{count}: {line.strip()}")
# in_f.close()

In [2]:
# read in train/test data
header_list = ['ID', 'Age', 'Gender', 'Education', 'Query_list']
nrows = None
df_train_data = pd.read_csv('data/train.csv', sep='###__###', engine='python', header=None, names=header_list, nrows=nrows)
df_train_data.head(2)

,ID,Age,Gender,Education,Query_list
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."


In [3]:
print(df_train_data.iloc[0]['Query_list'])
print(len(df_train_data.columns))
print(f'Number of training data: {len(df_train_data)}')

柔和双沟	女生	中财网首页 财经	http://pan.baidu.com/s/1plpjtn9	周公解梦大全查询2345	曹云金再讽郭德纲	总裁大人行行好	中财网第一财经传媒	教师节全文	男子砸毁15墓碑	黄岩岛最新填海图	引起的疲	缘来未迟落跑甜心不好惹	梁朝伟与替身同框	笑傲江湖电视剧任贤齐	小起名字女孩名字	海运行李到堪培拉	确定	诱爱99天 司少的天价宝贝	什么是遥控魔棒	徽信表情动态搞笑图片	教师节征文100字	安微联通网上营业厅	甜宠百分百:校草的萌萌未婚妻	豪门重生之暖爱成婚	nikehypershift和kd5哪个好看	韭菜炒鸡蛋	陈赫玩王者荣耀	虎牙楚河	三国演义小说txt下载	威县欧派	炒馍花怎么做好吃	黄岩岛最新消息2016年	中秋节诗句大全祝福	教师节征文	菜谱	柔和双沟卖的怎么样	七位数开奖结果	以色列停车场坍塌	天龙家庭农场	7.22什么星座	新旧约圣经和合本下载	4π	wifi万能钥匙	威灵仙图片	临泉长官天龙家庭农场	早安总统大人	百合	莲藕的做法	花街	无锡	蚬壳胃散怎么吃	触手忆寒	中秋节的诗句	孟州电信 电子发票	鸡丝汤的做法	我等你	临泉长官镇桥口李小刚农场	朋仇	全民k歌	炸葱花	蒜苔炒肉	冰川的图片	kd5	…	若风	好奇纸尿裤	清蒸鱼	189.8是谁的平方	重庆餐馆发生爆炸	捡手机被失主抢劫	https://yunpan.cn/ocsqfgtfya2ewj	炒馍花的家常做法	三国演义小说百度云	总裁掠爱小舅别太坏	:https://yunpan.cn/cmh8tmeyraiww	周公解梦	查坦克冰川	凉拌藕片的做法	投票	鸡丝炒什么好吃	被时光掩埋的秘密小说下载	中国电信电子发票	张续豪	关于月亮的诗句	用酵母蒸馒头的方法	赵丽颖碧瑶坐	触手兵长	图集 下载腾讯新闻,看街头混战武警	厦门航空	蚬壳胃散	炒茄子做法	身份类别怎么填	最好的我们里面的方特在哪里	牢里面的生活是怎样的	强迫症有哪些表现	白袍法师暖暖图片	朋仇广场舞	小宇热游	蒸馒头的方法	狡滑的意思	黄石大冶东岳派出所服务电话	三国演义小说下载txt	http://zxjhjc9088.1688.com	松柏道馆	10.1高速免费几天	三国演义小说txt	柔和双沟业务待遇	酵母蒸馒头的方法	初中家教一对一辅导	口子窖	中秋节祝福诗句	侠岚	文王国窖42度价格

In [4]:
stopwords=pd.read_csv("./external/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [5]:
import re
unused_tokens = ["http\S+", "\@\w+", "\#\w+", "www\S+"]

# remove noise
# invalid data
# - web link, e.g., http://pan.baidu.com/s/1plpjtn9
# - number sequence, e.g.,6933005900560
def remove_unused_text(document):
    noise_pattern = re.compile("|".join(unused_tokens))
    clean_text = re.sub(noise_pattern, "", document)
    return clean_text.strip()


def preprocess_text(age, gender, edu, document):
    document = remove_unused_text(document)
    processed_text = []
    sentences = document.split('\t')
#     print(document)
    for sen in sentences:
        try:
            segs=jieba.lcut(sen)
            segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的新闻过滤掉
            segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
#             print(segs)
            processed_text.extend(segs)
        except Exception as e:
            print(f'Exception happened due to {e}, as sentence={sen}')
            continue
    return " ".join(processed_text)
        

def remove_noise(df):
    new_queries = []
    for index, row in df.iterrows():
        age, gender, edu, query_list = row['Age'], row['Gender'], row['Education'], row['Query_list']
        new_query = preprocess_text(age, gender, edu, query_list)
        new_queries.append(new_query)
        if (index+1) % 2000 == 0:
            print(f'processed {index+1} rows, {(index+1)/len(df)} of data...')
    return new_queries

In [6]:
# process train data
new_queries = remove_noise(df_train_data)
df_train_data['Queries'] = new_queries
# print(len(new_queries))
# print(new_queries[0])

new_header = ['ID', 'Age', 'Gender', 'Education', 'Queries']
df_train_data.to_csv('data/processed_train.csv', columns=new_header)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/xn/j6h54rp17jncvq0ddzxntg240000gn/T/jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built successfully.


processed 2000 rows, 0.02 of data...
processed 4000 rows, 0.04 of data...
processed 6000 rows, 0.06 of data...
processed 8000 rows, 0.08 of data...
processed 10000 rows, 0.1 of data...
processed 12000 rows, 0.12 of data...
processed 14000 rows, 0.14 of data...
processed 16000 rows, 0.16 of data...
processed 18000 rows, 0.18 of data...
processed 20000 rows, 0.2 of data...
processed 22000 rows, 0.22 of data...
processed 24000 rows, 0.24 of data...
processed 26000 rows, 0.26 of data...
processed 28000 rows, 0.28 of data...
processed 30000 rows, 0.3 of data...
processed 32000 rows, 0.32 of data...
processed 34000 rows, 0.34 of data...
processed 36000 rows, 0.36 of data...
processed 38000 rows, 0.38 of data...
processed 40000 rows, 0.4 of data...
processed 42000 rows, 0.42 of data...
processed 44000 rows, 0.44 of data...
processed 46000 rows, 0.46 of data...
processed 48000 rows, 0.48 of data...
processed 50000 rows, 0.5 of data...
processed 52000 rows, 0.52 of data...
processed 54000 rows,

In [9]:
# df_test_data = pd.read_csv('data/test.csv', sep='###__###', engine='python', header=None, names=header_list, nrows=nrows)
# df_test_data.head(2)

# # process train/test data
# new_queries = remove_noise(df_test_data)
# df_test_data['Queries'] = new_queries
# print(len(new_queries))
# print(new_queries[0])

# new_header = ['ID', 'Age', 'Gender', 'Education', 'Queries']
# df_test_data.to_csv('data/processed_test.csv', columns=new_header)

,ID,Age,Gender,Education,Query_list
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,NaN,NaN,NaN
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,NaN,NaN,NaN
